<a href="https://colab.research.google.com/github/AliKarimiENT/Machine_Translation_EN_FA/blob/main/Machine_Translation_EN_FA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Translation Project